In [2]:
import numpy as np
import sys
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [79]:
d16 = pd.read_csv("data/properties_2016.csv")
t16 = pd.read_csv("data/train_2016.csv")
print(len(d16),len(t16))

2985217 90275


In [80]:
d17 = pd.read_csv("data/properties_2017.csv")
t17 = pd.read_csv("data/train_2017.csv")
print(len(d17),len(t17))

2985217 77613


In [69]:
table16 = pd.merge(d16,t16,on=["parcelid"],how="inner")
table17 = pd.merge(d17,t17,on=["parcelid"],how="inner")

In [70]:
assert len(table16) == len(t16)
assert len(table17) == len(t17)

In [73]:
data_table = pd.concat([table16,table17],axis = 0,ignore_index=True,join="outer")
print(f"{len(t16)}+{len(t17)}={len(data_table)}")
assert len(data_table) == len(t16)+len(t17)

90275+77613=167888


In [82]:
for col in data_table.columns:
    print(f"{col}: {str(data_table[col].dtype)}")
    u = data_table[col].unique()
    print(f"unique values: {len(u)} {u if len(u)<=20 else ''}")
    x = data_table[col].values
    x_not_null = data_table[col].notnull().values
    x_nn = x[x_not_null]
    print(f"{len(x_nn)} values are not null, covering {len(x_nn)/len(x):0.4} of all data.")
    print("")

parcelid: int64
unique values: 165210 
167888 values are not null, covering 1.0 of all data.

airconditioningtypeid: float64
unique values: 7 [nan  1.  5. 13. 11.  9.  3.]
53788 values are not null, covering 0.3204 of all data.

architecturalstyletypeid: float64
unique values: 7 [nan  7. 21.  8.  2.  3. 10.]
468 values are not null, covering 0.002788 of all data.

basementsqft: float64
unique values: 79 
93 values are not null, covering 0.0005539 of all data.

bathroomcnt: float64
unique values: 26 
167854 values are not null, covering 0.9998 of all data.

bedroomcnt: float64
unique values: 18 [ 3.  2.  4.  5.  6.  1.  0.  8. 10.  7. 12.  9. 13. 11. 15. 16. 14. nan]
167854 values are not null, covering 0.9998 of all data.

buildingclasstypeid: float64
unique values: 3 [nan  4.  3.]
31 values are not null, covering 0.0001846 of all data.

buildingqualitytypeid: float64
unique values: 13 [nan  4.  7. 10.  1.  8. 12.  6. 11.  9.  3.  5.  2.]
107173 values are not null, covering 0.6384 of 

In [76]:
#This cell turn all columns into int or float
data_table["hashottuborspa"] = data_table["hashottuborspa"].map(lambda x: 1. if x else 0.)
data_table = data_table.drop(["propertycountylandusecode","propertyzoningdesc","transactiondate"],axis=1)
data_table["hashottuborspa"] = data_table["hashottuborspa"].map(lambda x: 1. if x else 0.)
data_table["taxdelinquencyflag"] = data_table["taxdelinquencyflag"].map(lambda x: 1. if x=="Y" else 0.)
data_table["fireplaceflag"] = data_table["fireplaceflag"].map(lambda x: 1. if x else 0.)


In [77]:
corr = data_table.corr()

In [78]:
print(corr["logerror"])

parcelid                        0.010730
airconditioningtypeid           0.007077
architecturalstyletypeid       -0.006321
basementsqft                    0.324681
bathroomcnt                     0.027038
bedroomcnt                      0.028513
buildingclasstypeid             0.125978
buildingqualitytypeid          -0.007896
calculatedbathnbr               0.029432
decktypeid                           NaN
finishedfloor1squarefeet        0.007591
calculatedfinishedsquarefeet    0.039710
finishedsquarefeet12            0.043977
finishedsquarefeet13           -0.040595
finishedsquarefeet15           -0.008050
finishedsquarefeet50            0.003137
finishedsquarefeet6             0.049720
fips                            0.007406
fireplacecnt                    0.028223
fullbathcnt                     0.028079
garagecarcnt                    0.027544
garagetotalsqft                 0.029616
hashottuborspa                       NaN
heatingorsystemtypeid          -0.016296
latitude        

In [90]:
data_table.loc[data_table["rawcensustractandblock"].isnull(), "rawcensustractandblock"] = -1
data_table.loc[data_table["censustractandblock"].isnull(), "censustractandblock"] = -1
data_table[data_table["censustractandblock"]!=data_table["rawcensustractandblock"]][["censustractandblock","rawcensustractandblock"]]

,censustractandblock,rawcensustractandblock
0,6.111002e+13,6.111002e+07
1,6.111002e+13,6.111002e+07
2,6.111001e+13,6.111001e+07
3,6.111001e+13,6.111001e+07
4,6.111001e+13,6.111001e+07
...,...,...
167849,6.037190e+13,6.037190e+07
167850,6.059063e+13,6.059063e+07
167851,-1.000000e+00,6.059052e+07
167852,-1.000000e+00,6.059022e+07


In [91]:
data_table.loc[data_table["censustractandblock"]>0,"censustractandblock"].min()

60371011101000.0